In [ ]:
import pickle

def load_and_print_intent_stats(bin_path):
    # Load the pickle file
    with open(bin_path, "rb") as f:
        stats = pickle.load(f)
    
    print(stats['Rinku Singh'])
    # Print each batter and their metrics
    # for batter, metrics in stats.items():
    #     print(f"=== {batter} ===")
    #     for metric, value in metrics.items():
    #         print(f"{metric.replace('_', ' ').title()}: {value}")
    #     print()

if __name__ == "__main__":
    # adjust the path if needed
    paths = ["data/t20_intent_reliability_stats.bin","data/batter_intent_stats.bin","data/t20_batter_stats_360.bin","data/t20_entropy_focus.bin"]
    for p in paths:
        load_and_print_intent_stats(p)
    


{'pace_intent': 1.08, 'pace_reliability': 1.25, 'spin_intent': 0.99, 'spin_reliability': 1.15, 'pace_int_rel': 1.35, 'spin_int_rel': 1.14, 'avg_intent': 1.04, 'avg_reliability': 1.2, 'avg_int_rel': 1.25}
{'final_intent_impact': 13.733814943955947, 'impact_acceleration': 0.40393573364576313, 'negative_impact_duration': 1, 'impact_improvement': 0.6673045309261063, 'improvement_rate': 0.019626603850767834, 'improvement_pct': 48.484848484848484}
{'score360': np.float64(355.12), 'ground_coverage_pct': np.float64(39.4), 'audacity': np.float64(4.71), 'aggressive_bp_pct': 25.6, 'overall_bp_pct': 22.1, 'diff_shot_effect': 1.16}
{'primary_focus': 'Bowler Kind', 'least_affected_by': 'Field Changes'}
1


In [ ]:
import pickle
import pandas as pd

# map the same section names used in your app to their .bin paths
SECTIONS = {
    "Intent & Reliability":      "data/t20_intent_reliability_stats.bin",
    "Intent Impact Metrics":     "data/batter_intent_stats.bin",
    "360° Shot Metrics":         "data/t20_batter_stats_360.bin",
}

percentiles = {}

for section, path in SECTIONS.items():
    with open(path, "rb") as f:
        data = pickle.load(f)

    # build DataFrame: rows=batter, cols=metrics
    df = pd.DataFrame.from_dict(data, orient="index")
    
    # keep only numeric columns
    num_df = df.select_dtypes(include="number")
    if num_df.empty:
        continue

    # compute percentile rank (0–100) for each column
    pct_df = num_df.rank(pct=True, method="average") * 100

    # invert negative_impact_duration for Intent Impact Metrics
    if section == "Intent Impact Metrics" and "negative_impact_duration" in pct_df.columns:
        # higher percentiles should correspond to smaller durations
        pct_df["negative_impact_duration"] = (100 - pct_df["negative_impact_duration"])

    # build nested dict: percentiles[section][batter][metric] = percentile
    pct_map = {
        batter: pct_df.loc[batter].to_dict()
        for batter in pct_df.index
    }
    percentiles[section] = pct_map

# save the combined percentile map
with open("data/metric_percentiles.bin", "wb") as f:
    pickle.dump(percentiles, f)

print("Saved metric percentiles → data/metric_percentiles.bin")



Saved metric percentiles → data/metric_percentiles.bin


In [10]:
print(percentiles)

{'Intent & Reliability': {'Omid Shafi Rahman': {'pace_intent': 0.31982942430703626, 'pace_reliability': 6.023454157782516, 'spin_intent': 0.7995735607675907, 'spin_reliability': 0.7196162046908315, 'pace_int_rel': 0.31982942430703626, 'spin_int_rel': 0.7995735607675907, 'avg_intent': 0.15991471215351813, 'avg_reliability': 0.31982942430703626, 'avg_int_rel': 0.15991471215351813}, 'Yash Dayal': {'pace_intent': 0.31982942430703626, 'pace_reliability': 8.448827292110876, 'spin_intent': 0.7995735607675907, 'spin_reliability': 0.26652452025586354, 'pace_int_rel': 0.31982942430703626, 'spin_int_rel': 0.7995735607675907, 'avg_intent': 0.15991471215351813, 'avg_reliability': 0.39978678038379534, 'avg_int_rel': 0.15991471215351813}, 'Mohammad Zeeshan': {'pace_intent': 0.31982942430703626, 'pace_reliability': 25.5863539445629, 'spin_intent': 0.7995735607675907, 'spin_reliability': 2.398720682302772, 'pace_int_rel': 0.31982942430703626, 'spin_int_rel': 0.7995735607675907, 'avg_intent': 0.15991471